In [8]:
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from skmob.utils import utils
from math import pi, asin, sin, cos, sqrt

pd.set_option('display.max_columns', 500)



%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
plt.rcParams["figure.figsize"] = (12, 9) # (w, h)

In [9]:
def distance_earth_km(src, dest):

    lat1, lat2 = src['lat']*pi/180, dest['lat']*pi/180
    lon1, lon2 = src['lon']*pi/180, dest['lon']*pi/180
    dlat, dlon = lat1-lat2, lon1-lon2

    ds = 2 * asin(sqrt(sin(dlat/2.0) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2.0) ** 2))
    return 6371.01 * ds

In [12]:
def filter_tessellation_land(tessellation, shape_file_land):    
    tiles_in_land = gpd.sjoin(tessellation, shape_file_land, how='left', op='intersects')
    tiles_in_land = tiles_in_land.groupby(['tile_ID'],sort=False,as_index=False).first()
    #land = tiles_in_land.dropna().drop(["index_right","boro_code","boro_name","shape_area","shape_leng"],axis=1)
    #water = tiles_in_land[tiles_in_land['index_right'].isnull()].drop(["index_right","boro_code","boro_name","shape_area","shape_leng"],axis=1)    
    
    land = tiles_in_land.dropna()[['tile_ID', 'geometry']]
    water = tiles_in_land[tiles_in_land['index_right'].isnull()][['tile_ID', 'geometry']]     
    
    crs = {'init': 'epsg:4326'}    
    land = gpd.GeoDataFrame(land, crs=crs, geometry='geometry')
    water = gpd.GeoDataFrame(water, crs=crs, geometry='geometry')     
    return {"land":land, "water":water}

In [10]:
def compute_distance_matrix(tess):

    lats_lngs = tess.geometry.apply(utils.get_geom_centroid, args=[True]).values
    
    distance_matrix = np.zeros((len(tess),len(tess)))

    for i in range(0,len(tess)):
        for j in range(0,len(tess)):
            if i != j:
                d = distance_earth_km({'lat':lats_lngs[i][0],'lon':lats_lngs[i][1]},
                                         {'lat':lats_lngs[j][0],'lon':lats_lngs[j][1]})
                distance_matrix[i,j] = d
                
    return distance_matrix

In [26]:
meters = 1840
tesselletion = tilers.tiler.get("squared", meters=meters, base_shape="New York City")
tesselletion


,tile_ID,geometry
0,0,"POLYGON ((-74.25909 40.47740, -74.25909 40.489..."
1,1,"POLYGON ((-74.25909 40.48997, -74.25909 40.502..."
2,2,"POLYGON ((-74.25909 40.50254, -74.25909 40.515..."
3,3,"POLYGON ((-74.25909 40.51511, -74.25909 40.527..."
4,4,"POLYGON ((-74.25909 40.52767, -74.25909 40.540..."
...,...,...
698,698,"POLYGON ((-73.73016 40.76595, -73.73016 40.778..."
699,699,"POLYGON ((-73.71363 40.71586, -73.71363 40.728..."
700,700,"POLYGON ((-73.71363 40.72839, -73.71363 40.740..."
701,701,"POLYGON ((-73.71363 40.74091, -73.71363 40.753..."


In [28]:
plot_gdf(tesselletion, style_func_args={'fillColor':'gray', 'color':'black', 'opacity': 0.2}, zoom = 9) 

In [29]:
shape_file_land = gpd.read_file("NYC_shapeNoWaterArea.geojson")
shape_file_land_MAN = shape_file_land.iloc[[4]]
shape_file_land_MAN

,boro_code,boro_name,shape_area,shape_leng,geometry
4,1,Manhattan,636602658.764,361212.476577,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."


In [30]:
res_inter = filter_tessellation_land(tesselletion, shape_file_land_MAN )
tess_nyc = res_inter['land']
tess_nyc

/opt/anaconda3/envs/skmob/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,tile_ID,geometry
154,154,"POLYGON ((-74.06074 40.67827, -74.06074 40.690..."
155,155,"POLYGON ((-74.06074 40.69080, -74.06074 40.703..."
170,170,"POLYGON ((-74.04421 40.67827, -74.04421 40.690..."
171,171,"POLYGON ((-74.04421 40.69080, -74.04421 40.703..."
186,186,"POLYGON ((-74.02768 40.67827, -74.02768 40.690..."
...,...,...
358,358,"POLYGON ((-73.92851 40.85353, -73.92851 40.866..."
359,359,"POLYGON ((-73.92851 40.86603, -73.92851 40.878..."
360,360,"POLYGON ((-73.92851 40.87853, -73.92851 40.891..."
394,394,"POLYGON ((-73.91198 40.86603, -73.91198 40.878..."


In [31]:
plot_gdf(tess_nyc, style_func_args={'fillColor':'gray', 'color':'black', 'opacity': 0.2}, zoom = 9) 

In [38]:
len(tess_nyc)

64

In [39]:
dist = compute_distance_matrix(tess_nyc)
dist

array([[ 0.        ,  1.39360087,  1.39373193, ..., 24.8943093 ,
        24.34594961, 25.54700728],
       [ 1.39360087,  0.        ,  1.97084925, ..., 23.6555526 ,
        23.16140918, 24.3413622 ],
       [ 1.39373193,  1.97084925,  0.        , ..., 24.30384693,
        23.66001076, 24.8943093 ],
       ...,
       [24.8943093 , 23.6555526 , 24.30384693, ...,  0.        ,
         1.96528619,  1.38953601],
       [24.34594961, 23.16140918, 23.66001076, ...,  1.96528619,
         0.        ,  1.38966719],
       [25.54700728, 24.3413622 , 24.8943093 , ...,  1.38953601,
         1.38966719,  0.        ]])

In [40]:
np.save("dist_mat.npy", dist)